# PS4: Let's build a Personal Shopper
In this problem set, we'll build a simple personal shopper program that helps users find items they might want to buy based on their preferences. The program will allow users to input their preferences and then suggest items from a predefined list that maximize their satisfaction (measured using a utility function) based on those preferences, subjet to a budget constraint.

## Task 1: Setup, Data, and Prerequisites
We set up the computational environment by including the `Include.jl` file, loading any needed resources, such as sample datasets, and setting up any required constants. 
* The `Include.jl` file also loads external packages, various functions that we will use in the exercise, and custom types to model the components of our problem. It checks for a `Manifest.toml` file; if it finds one, packages are loaded. Other packages are downloaded and then loaded.

In [1]:
include("Include.jl"); # This will load necessary packages and functions

First, let's build the `world(...)` function. 
* The `world(...)` function takes the $m$-dimensional action vector `a::Array{Int64,1}` where the elements of `a::Array{Int64,1}` are the indexes of the goods chosen from each categories, the amount of each good selected from each category from our agent is in the `n::Dict{Int,Array{Float64,1}}` dictionary, and returns the reward (utility) $r\sim\mathcal{D}_{a}$. associated with selecting this action. 

In [2]:
function world(a::Vector{Int64}, n::Array{Float64,1}, context::MyBanditConsumerContextModel)::Float64

    # initialize -
    γ = context.γ; # consumer preferences (unknown to bandits)
    σ = context.σ; # noise in utility calculation (unknown to bandits)
    B = context.B; # max budget (unknown to bandits)
    C = context.C; # unit costs of goods (unknown to bandits)
    λ = context.λ; # sensitivity to the budget
    Z = context.Z; # noise model
    number_of_goods = context.m; # number of categories

    # compute the reward for this choice -
    Ū = 1.0;
    BC = 0.0;
    for i ∈ 1:number_of_goods
        
        # what action is being taken in this category?
        aᵢ = a[i]; # this is which good to purchase in category i -
        if (aᵢ == 0)
            # if aᵢ is 0, it means no good was chosen in this category, 
            # hence we should skip this category in the utility calculation
            continue; 
        end

        nᵢ = n[i]; # this is the quantity purchased of good aᵢ in category i
        Cᵢ = C[i]; # cost of chosen good in category i
        γᵢ = γ[i]; # preference of good in category i
        σᵢ = σ[i]; # standard dev for good i
   
        # update the utility -
        Ū += γᵢ*(nᵢ + σᵢ*rand(Z)); # compute the utility for this good, with noise
        
        # constraints and noise 
        BC += nᵢ*Cᵢ; # compute the budget constraint -
    end

    # compute the budget constraint violation -
    U = Ū - λ*max(0.0, (BC - B))^2; # use a penalty method to capture budget constraint

    # return the reward -
    return U;
end;

Next, let's build an algorithm model that we'll use to reason about the world, i.e., a model of our agent's decision making process.  we've modified the $\epsilon$-greedy algorithm to work with our contextual category-based bandit problem. The algorithm will now take into account the context provided by [the `MyBanditConsumerContextModel` model](src/Types.jl) and category-based actions when selecting actions.

#### Epsilon-Greedy with Categories Algorithm
In the _epsilon-greedy_ algorithm, the agent chooses the best arm with probability $1-\epsilon$ and a random arm with probability $\epsilon$. This approach balances exploration and exploitation by allowing the agent to explore different arms while also exploiting the best-known arm based on past rewards. The parameter $\epsilon$ controls the exploration rate: a higher value means more exploration, while a lower value means more exploitation.

* While [Slivkins](https://arxiv.org/abs/1904.07272) doesn't give a reference for the $\epsilon$-greedy algorithm, other sources point to (at least in part) to [Thompson and Thompson sampling, proposed in 1933 in the context of drug trials](https://arxiv.org/abs/1707.02038). Thus, the $\epsilon$-greedy algorithm, considered a classic algorithm in the multi-armed bandit literature. The algorithm is simple yet effective, making it a popular choice for many practical applications.

The agent has $K$ arms (choices), $\mathcal{A} = \left\{1,2,\dots,K\right\}$, where each arm encosed a _binary vector_ which describes the _basket_ of goods chosen by the agent, 
and the total number of rounds is $T$. The agent uses the following algorithm to choose which arms to pull (which action to take) during each round:

For $t = 1,2,\dots,T$:
1. _Initialize_: Roll a random number $p\in\left[0,1\right]$ and compute a threshold $\epsilon_{t}\sim{t}^{-1/3}$.
2. _Exploration_: If $p\leq\epsilon_{t}$, choose a random (uniform) arm $a_{t}\in\mathcal{A}$. Execute the action $a_{t}$ and receive a reward $r_{t}$ from the _adversary_ (nature). 
3. _Exploitation_: Else if $p>\epsilon_{t}$, choose action $a^{\star}$, the action with the highest average reward so far. Execute the action $a^{\star}_{t}$ and recieve a reward $r_{t}$ from the _adversary_ (nature).
4. Update list of rewards for $a_{t}\in\mathcal{A}$

__Theorem__: The epsilon-greedy algowithm with exploration probability $\epsilon_{t}={t^{-1/3}}\cdot\left(K\cdot\log(t)\right)^{1/3}$ achives a regret bound of $\mathbb{E}\left[R(t)\right]\leq{t}^{2/3}\cdot\left(K\cdot\log(t)\right)^{1/3}$ for each round $t$.

Let's build [a `MyEpsilonGreedyAlgorithmModel` instance](src/Types.jl) which encapsulates the $\epsilon$-greedy logic that has been modified to work with categories. To contstruct this type, we [use a custom `build(...)` method](src/Factory.jl) that takes the arms dictionary `K::Dict{Int64,Int64}` and the items dictionary `n::Dict{Int64, Array{Float64,1}}` and returns an algorithmn model in the `algorithm::MyEpsilonGreedyAlgorithmModel` variable.

In [3]:
algorithm = let

    # initialize -
    K = 12; # suppose we have 12 possible goods that we can choose from
    n = ones(Float64, K); # for now, let's assume that we purchase a single unit of each good in each category (we can change this later)
    
    # build model -
    algorithm = build(MyEpsilonGreedyAlgorithmModel, (
        K = K, # arms dictionary
        n = n, # items dictionary
    ));

    # return the algorithm -
    algorithm;
end;

__Constants__: Let's set some constants we'll use in the subsequent tasks. See the comment beside the value for a description of what it is, its permissible values, etc.

In [ ]:
T = 10; # number of rounds for each decision task
B = 100.0; # Budget for shopper

## Task 2: Build the Context Models
In this task, we will build several models of the contextual information that will be used to inform the agent's recomendations. These models, which are [instances of the `MyBanditConsumerContextModel` type](src/Types.jl) holds various parameters that will be used in the `world(...)` funtion that we developed above. 
* _Hmmm. Why use a different model for contextual data_? We use a separate model for the contextual information because it allows us to encapsulate all the relevant parameters and settings in one place. This makes it easier to manage and modify the parameters as needed, without having to change the core logic of the `world(...)` function. Additionally, it allows us to easily pass around context models to other parts of our codebase that may need it.
* _What does this represent_? The contextual information in the `MyBanditConsumerContextModel` represents the parameters that will be used to score the utility of the goods chosen by the agent. This includes user sentiment parameters, budget constraints, and other relevant information that will help the agent make informed decisions about which goods to recommend.

Let's build the following contextual models:
* __Case 1: Unlimited budget, uniform positive sentiment__: This model assumes that the consumer has an unlimited budget ($\lambda = 0$) and uniform positive sentiment across all goods. This means that the consumer is equally likely to choose any good in each category, and there are no constraints on the amount of each good that can be selected. 
* __Case 2: Limited budget, positive sentiment__: This model assumes that the consumer has a limited budget $\lambda>0$ and positive sentiment (but not neccesarily uniform) towards goods. This means that the consumer is more likely to choose goods that they have a positive sentiment towards, and there are constraints on the amount of each good that can be selected based on the budget.
* __Case 3: Limited budget, mixed sentiment__: This model assumes that the consumer has a limited budget $\lambda>0$ and mixed sentiment towards goods. This means that some goods may have positive sentiment (i.e., they are preferred), while others may have negative sentiment (i.e., they are not preferred). The agent must balance the positive and negative sentiments when making recommendations, and there are constraints on the amount of each good that can be selected based on the budget.

In [5]:
simple_no_budget_context = let

    # initialize -
    K = algorithm.K; # number of arms in the algorithm, this should match the number of goods in the context model
    γ = Array{Float64,1}(undef, K); # consumer preferences (unknown to bandits)
    σ = Array{Float64,1}(undef, K); # noise in utility calculation (unknown to bandits)
    C = Array{Float64,1}(undef, K); # unit costs of goods (unknown to bandits)
    Z = Normal(0,1); # use a standard normal distribution for the noise model, this can be changed to any distribution as required
    λ = 0.0; # sensitivity to the budget constraint λ ≥ 0. If zero, then no penalty for budget constraint violation.

    # set the parameters -
    # preferences: If all γ[i] are equal to 1.0, then the bandit will be indifferent to the goods in each category.
    for i ∈ 1:K
        # Assigning values for γ, σ, and C for each good in the context model
        # For simplicity, let's assume we have K goods with equal preference
        # This can be customized as per the requirement of the simulation
        γ[i] = 1.0; # uniform preference for all goods
        σ[i] = 0.1; # uniform uncertainty for all goods, this can be adjusted based on the specific needs of the simulation
        C[i] = 10.0 + 10.0 * (i - 1); # linearly increasing costs for goods, this can be customized as per the requirement
    end

    # build a context model with the reqired parameters -
    context = build(MyBanditConsumerContextModel, (
        γ = γ, # consumer preferences (unknown to bandits)
        σ = σ, # noise in utility calculation (unknown to bandits)
        B = B, # max budget (unknown to bandits)
        C = C, # unit costs of goods (unknown to bandits)
        λ = λ, # sensitivity to the budget
        Z = Z, # noise model
        m = K, # number of categories (this should match the number of arms in the algorithm)
    )); # build the context

    # return 
    context;
end;

## Task 3: Evaluation of Scenarios
In this task, we'll run different context models to evaluate how well our agent performs under different scenarios. This will help us understand how the choice of context model affects the agent's performance and the regret it incurs over time. 

In all cases, we will use the same agent and bandit algorithm but vary the context model to see how it influences the agent's decisions and performance. We'll use the $\epsilon$-greedy algorithm to estimate an optimal policy for the agent based on the contextual information provided by the `MyBanditConsumerContextModel`.

In [ ]:
results_case_1 = solve(algorithm, T = T, world = world, context=simple_no_budget_context);